In [3]:
import numpy as np
import sys
sys.path.append("..")
from reference.lib.envs.gridworld import GridworldEnv

In [4]:
env = GridworldEnv()

In [37]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.0000001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    # loop until convergence - system is stable
    while True:
        # this is the change value
        delta_value = 0
        
        # for every state value
        for i in range(len(V)):
            
            current_state = V[i]
            V[i] = 0
            
            for policy_index in range(4):
                action = policy_index
                action_prob = policy[i,policy_index]
                
                for prob, next_state, reward, done in env.P[i][action]:
                    V[i] = V[i] + action_prob * prob * (reward + discount_factor * V[next_state]) 
                    
            delta_value = max(delta_value,np.abs(current_state-V[i]))
            #V[i] = current_state
            
        if delta_value < theta: 
            break
            
    return np.array(V)

In [36]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)
print(v.reshape((4,4)))

[[ 0.         -4.0125803  -4.65069603 -3.4864739 ]
 [-5.99713157 -7.39963785 -7.10373846 -5.0452027 ]
 [-7.99120121 -8.48511617 -7.31942619 -4.46898239]
 [-7.2896119  -7.23021121 -5.21987349  0.        ]]


In [32]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
print(expected_v.reshape((4,4)))

[[  0 -14 -20 -22]
 [-14 -18 -20 -20]
 [-20 -20 -18 -14]
 [-22 -20 -14   0]]


In [33]:
print(expected_v/(v+1e-8))

[0.         1.00000462 1.00000465 1.00000465 1.00000462 1.00000441
 1.00000431 1.00000426 1.00000465 1.00000431 1.00000404 1.00000388
 1.00000465 1.00000426 1.00000388 0.        ]


In [34]:
# test 
print(
    np.testing.assert_array_almost_equal(v, expected_v, decimal=2)
)

None
